# **Phenotype Prediction: Days to Flowering**

The phenotype of an organism is the result of an interplay betweeen its genetic composition and the environment. Due to the gradual change of the climate, it becomes essential to understand the influence of genotype and the environment on plant phenotypes. Here, we intend to develop a machine learning (ML) model, which could predict the phenotype: Days to Flowering. Currently, our model can predict the phenotypic attributes in Sorghum bicolor and we intend to solve this problem in other plant species as well. The model takes the genetic data of the plants and environmental data as input. The gene data is based on Single Nucleotide Polymorphisms (SNP). It is filtered by entropy and added as features in the model. The environmental data consists of location of the plant (range and column of the subplot) and weather parameters from the weather station. Also, sowing date of the plants and window size are required to give as input parameter to the model. We use Extreme Gradient Boosting (XgBoost) regressor to model the data with 5-fold cross-validation and the script outputs a trained model which can be used to predict the phenotype.    

**Dataset:**
The following dataset is required in a particular format.

**1. Trait Data of Plant Species:** Each observation in this dataset denotes the location (range and column) of the subplot, cultivar information, the date and days of flowering.

*Columns: plot, range, column, scientificname, genotype, treatment, blocking_height, method, date_of_flowering, days_to_flowering, gdd_to_flowering, method_type*

**2. Gene Data Filtered by Entropy:** The gene data comprises of SNPS from 4.4k genes present in 362 cultivars. The gene data is clustered using k-means with Euclidean distances for 30 gene clusters. Thus, the number of SNPs that each cultivar has in 30 gene clusters is added as features in the model.

**3. Weather Data:** The weather data is obtained from a local weather station and the parameters include temperature, relative humidity, vapor pressure deficit, growing degree days and precipitation.

*Columns: date, day_of_year, temp_min, temp_max, temp_mean, gdd, rh_min, rh_max, rh_mean, vpd_mean, precip, precip_cumulative, first_water_deficit_treatment, second_water_deficit_treatment*

**4. Sowing Date:** The date at which the seeds of the plants were sowed.

**5. Window Size:** The number of days from the sowing date for which the environmental variables are required to be used to train the model.




The following script needs to be executed. In the first code block, neccesary user input is required. The cluster file (gene data filtered by entropy) is also required to be present with this python notebook. 

In [ ]:
# ***User input is required***
# - Specify the URL/path of the trait data needs to be imported and the corresponding delimeter
url_trait = "/content/drive/MyDrive/GPE/MAC Season 4/mac_season_4_days_gdd_to_flowering.csv"
delim_trait = ","
# - Specify the URL/path of the weather data needs to be imported and the corresponding delimeter
weather_url = "/content/drive/MyDrive/GPE/weather_data/mac_season_4_weather.csv"
weather_delim = ','
# - Specify the sowing date
sowing_date = "2017-04-20"
# - Specify the window size
window_size = 80

In [ ]:
#importing the relevant python packages
import pandas as pd
import seaborn as sns
import numpy as np
import math
from scipy import stats
import statistics
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

In [ ]:
# This code block is not required to be executed if any github URL or a path on local machine is being provides as the path for the data. 
# Since, currently we are using Google Colab notebook and data is store on Google Drive, hence the drive location is mounted over here. 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing the data 
fl = pd.read_csv(url_trait , delimiter = delim_trait)
fl2=fl[['days_to_flowering', 'cultivar', 'range', 'column']]

# Write about dist
gene = pd.read_csv("/content/drive/MyDrive/GPE/Cluster.txt", delimiter="\t")
a = gene.X
gene = gene.rename(columns={"X": "cultivar"})
gene = gene.dropna()

fl2=pd.merge(fl2, gene, on="cultivar", how='inner')

In [ ]:
# Import the weather data and calculate the average of weather paramaters between the sowing date and the window size (since weather data is at day level).
mac=pd.read_csv(weather_url , delimiter= weather_delim)
mac['date']=pd.to_datetime(mac.date)
day_0=pd.to_datetime(sowing_date)
day_n=day_0 + pd.to_timedelta(window_size, 'days')

#for mean
tmax = []
tmean=[]
tmin = []
rhmax=[]
rhmin=[]
rhmean=[]
vpd=[]
 
for j in range(fl2.shape[0]):
    li = (mac['date']>= day_0) & (mac['date'] <= day_n)
    df_1 = mac[li]
    tmax.append(df_1['temp_max'].mean())
    tmin.append(df_1['temp_min'].mean())
    tmean.append(df_1['temp_mean'].mean())
    rhmax.append(df_1['rh_max'].mean())
    rhmin.append(df_1['rh_min'].mean())
    rhmean.append(df_1['rh_mean'].mean())
    vpd.append(df_1['vpd_mean'].mean())

fl2.insert(1,'temp_max_mean',tmax)
fl2.insert(2,'temp_min_mean',tmin)
fl2.insert(3,'temp_mean_mean',tmean)
fl2.insert(4,'rh_max_mean',rhmax)
fl2.insert(5,'rh_min_mean',rhmin)
fl2.insert(6,'rh_mean_mean',rhmean)
fl2.insert(7,'vpd_mean',vpd)

#for maximum
tmax = []
tmean=[]
tmin = []
rhmax=[]
rhmin=[]
rhmean=[]
vpd=[]


for j in range(fl2.shape[0]):
    li = (mac['date']>= day_0) & (mac['date'] <= day_n)
    df_1 = mac[li]
    tmax.append(df_1['temp_max'].max())
    tmin.append(df_1['temp_min'].max())
    tmean.append(df_1['temp_mean'].max())
    rhmax.append(df_1['rh_max'].max())
    rhmin.append(df_1['rh_min'].max())
    rhmean.append(df_1['rh_mean'].max())
    vpd.append(df_1['vpd_mean'].max())


fl2.insert(1,'temp_max_max',tmax)
fl2.insert(2,'temp_min_max',tmin)
fl2.insert(3,'temp_mean_max',tmean)
fl2.insert(4,'rh_max_max',rhmax)
fl2.insert(5,'rh_min_max',rhmin)
fl2.insert(6,'rh_mean_max',rhmean)
fl2.insert(7,'vpd_max',vpd)

#for minimum
tmax = []
tmean=[]
tmin = []
rhmax=[]
rhmin=[]
rhmean=[]
vpd=[]

for j in range(fl2.shape[0]):
    li = (mac['date']>= day_0) & (mac['date'] <= day_n)
    df_1 = mac[li]
    tmax.append(df_1['temp_max'].min())
    tmin.append(df_1['temp_min'].min())
    tmean.append(df_1['temp_mean'].min())
    rhmax.append(df_1['rh_max'].min())
    rhmin.append(df_1['rh_min'].min())
    rhmean.append(df_1['rh_mean'].min())
    vpd.append(df_1['vpd_mean'].min())

fl2.insert(1,'temp_max_min',tmax)
fl2.insert(2,'temp_min_min',tmin)
fl2.insert(3,'temp_mean_min',tmean)
fl2.insert(4,'rh_max_min',rhmax)
fl2.insert(5,'rh_min_min',rhmin)
fl2.insert(6,'rh_mean_min',rhmean)
fl2.insert(7,'vpd_min',vpd)

gdd=[]
pre=[]
cpre=[]
#for sum
for j in range(fl2.shape[0]):
    li = (mac['date']>= day_0) & (mac['date'] <= day_n)
    df_1 = mac[li]
    gdd.append(df_1['gdd'].sum())
    pre.append(df_1['precip'].sum())
    cpre.append(df_1['precip_cumulative'].sum())

fl2.insert(8,'gdd_sum',gdd)
fl2.insert(8,'precip_sum',pre)
fl2.insert(8,'cumprecip_sum',cpre)

In [ ]:
new=fl2.drop(['cultivar', 'days_to_flowering'], axis=1)
X=new.to_numpy()
min_max_scaler = MinMaxScaler()

y=fl2[['days_to_flowering']]
y = np.asarray(y).flatten()

In [ ]:
# Parameter tuning and 5 fold cross-validation for Extreme Gradient Boosting (XgBoost) Regression
ns=5
groups=fl2.cultivar
cv =GroupKFold(n_splits=ns)

model = xgb.XGBRegressor()
groups=fl2.cultivar
n_estimators = range(10, 60, 5)
learning_rate = np.linspace(0,1,11)
max_depth = range(1, 20, 2)
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

cv =GroupKFold(n_splits=ns)
error=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=xgb.XGBRegressor(booster='gbtree',importance_type='gain',learning_rate= 0.2, max_depth= 3, n_estimators= 20).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    
std= (statistics.stdev(e))

print('Root Mean square error for 5-fold CV XG:',error/ns, '+-', std)


[21:56:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best: -10.520522 using {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 20}
[21:56:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:56:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:56:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:56:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:56:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Root Mean square error for 5-fold CV XG: 10.520522366932623 +- 1.621935718989048
